In [1]:
!pip install -U transformers
!pip install -U accelerate
# !pip install unidecode
# !pip install datasets
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

# Import

In [2]:
import numpy as np
import pandas as pd
import pickle
import math
import string

In [3]:
# import datasets
# from datasets import Dataset, load_metric

In [4]:
from transformers import TextDataset, DataCollatorForLanguageModeling
# from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [5]:
# from bs4 import BeautifulSoup
# import unidecode
# import spacy
# nlp = spacy.load("en_core_web_sm")
# import re

import nltk
# from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
# nltk.download('wordnet')
# lemmatizer = WordNetLemmatizer()

In [6]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import AutoTokenizer, AutoModelWithLMHead
# Load tokenizer from Hugging face
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = AutoModelWithLMHead.from_pretrained('gpt2')
model.config.pad_token_id = model.config.eos_token_id
max_length = 128

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1699: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
# Vectorizer
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.naive_bayes import GaussianNB
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report
# from sklearn.model_selection import KFold

# Data Read


In [15]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
df_reviews = pd.read_csv("/content/drive/MyDrive/Reviews.csv")

In [ ]:
print(df_reviews.shape)
df_reviews.drop(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time'], inplace = True, axis = 1)
df_reviews.drop_duplicates(inplace = True)
print(df_reviews.shape)

(568454, 10)
(394970, 2)


In [ ]:
df_reviews.columns

Index(['Summary', 'Text'], dtype='object')

In [ ]:
df_reviews.head()

,Summary,Text
0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,"""Delight"" says it all",This is a confection that has been around a fe...
3,Cough Medicine,If you are looking for the secret ingredient i...
4,Great taffy,Great taffy at a great price. There was a wid...


#Pre-processing

In [ ]:
def pre_process_text(query):
  try:
    query.lower()
  except:
    print(query)
  # Convert the query to lowercase
  content = query.lower()

  # Tokenize the query
  tokens = nltk.word_tokenize(content)

  # Remove stop words
  stop_words = set(stopwords.words('english'))
  filtered_tokens = [token for token in tokens if token not in stop_words]

  # Join the filtered tokens back into a string
  content_without_stopwords = ' '.join(filtered_tokens)

  # Remove punctuation using string.punctuation
  cleaned_content = ''.join(char for char in content_without_stopwords if char not in string.punctuation)

  # Remove blank_spaces using string.blank_spaces
  final_query = ' '.join(char for char in cleaned_content.split() if char)
  return final_query

In [ ]:
df_rev = df_reviews.sample(frac = 0.1, random_state  = 1)

NameError: name 'df_reviews' is not defined

In [ ]:
df_rev.shape

(3950, 2)

In [ ]:
df_rev["Summary"] = df_rev["Summary"].fillna("")
# df_reviews["Summary"] = df_reviews["Summary"].replace('', np.nan)
# df_reviews["Summary"].dropna(inplace = True)

In [ ]:
df_rev["Summary"] = df_rev["Summary"].apply(pre_process_text)

In [ ]:
df_rev["Text"] = df_rev["Text"].fillna("")
# df_rev["Text"] = df_rev["Text"].dropna()
df_rev["Text"] = df_rev["Text"].apply(pre_process_text)

In [ ]:
with open("/content/drive/MyDrive/df_rev.pickle", 'wb') as file:
  pickle.dump(df_rev, file)

In [9]:
with open("/content/drive/MyDrive/df_rev.pickle", 'rb') as file:
  df_rev = pickle.load(file)

In [10]:
df_train, df_val = train_test_split(df_rev, test_size=0.25, random_state=42)

In [ ]:
df_train.shape

(2962, 3)

In [11]:
df_train['training'] = df_train['Text']  + ' TL;DR ' + df_train['Summary']

In [ ]:
df_rev.shape

(3950, 3)

# Dataset Creation


## Try 4

In [12]:
import torch
from torch.utils.data import Dataset

class TextDataset(Dataset):
  """Custom dataset class for text data."""

  def __init__(self, df, column_name):
    """
    Initialize the dataset with the DataFrame and column name.

    Args:
      df (pandas.DataFrame): The DataFrame containing your data.
      column_name (str): The name of the column containing the text data.
    """
    self.data = df[column_name].tolist()  # Extract text data as a list

  def __len__(self):
    """Return the total number of samples in the dataset."""
    return len(self.data)

  def __getitem__(self, idx):
    """Retrieve a single data sample at the specified index.

    Args:
      idx (int): Index of the sample to retrieve.

    Returns:
      str: The text data at the specified index.
    """
    return self.data[idx]

# Create the dataset
dataset = TextDataset(df_train, "training")

In [ ]:
dataset[:5]

['love pizza crust mix make pizza weekly crisp doughy make vegan using ener g egg replacer comes great every time TL;DR great pizza mix',
 'second time ordered since great price dog loves however make mistake odor free seems worse smell dog loves tho price ca nt beat shipped extremely fast great seller TL;DR great product smells horrible',
 'bought product curiosity small recipe book came since idea use mix used recipe book chicken sauerkraut goulash tasted good pretty mush use mix recipe since may taste like typical goulash product really love TL;DR love taste',
 'dogs love tasty little treats 3 calories great training tool bit disapointed got liver order bag liver bag cheese TL;DR good training tool dog',
 'living england getting 2 less per conversion hate pay 299 per states baked beans really like also comfort food br br thank amazon selling product bulk TL;DR thank product']

In [13]:
class GPT2ReviewDataset(Dataset):
    def __init__(self, tokenizer, reviews, max_len):
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.eos = self.tokenizer.eos_token
        self.eos_id = self.tokenizer.eos_token_id
        self.reviews = reviews
        self.result = []

        for review in self.reviews:
            # Encode the text using tokenizer.encode(). We add EOS at the end
            tokenized = self.tokenizer.encode(review + self.eos)

            # Padding/truncating the encoded sequence to max_len
            padded = self.pad_truncate(tokenized)

            # Creating a tensor and adding to the result
            self.result.append(torch.tensor(padded))

    def __len__(self):
        return len(self.result)


    def __getitem__(self, item):
        return self.result[item]

    def pad_truncate(self, name):
        extra_length = len(tokenizer.encode(" TL;DR "))
        name_length = len(name) - extra_length
        if name_length < self.max_len:
            difference = self.max_len - name_length
            result = name + [self.eos_id] * difference
        elif name_length > self.max_len:
            result = name[:self.max_len + 3]+[self.eos_id]
        else:
            result = name
        return result

In [14]:
train_dataset = GPT2ReviewDataset(tokenizer, dataset, max_length)

In [ ]:
def train(train_dataset, tokenizer, model,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):

    data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)

    tokenizer.save_pretrained(output_dir)

    model.save_pretrained(output_dir)

    training_args = TrainingArguments(
        output_dir='gpt_rev',
        overwrite_output_dir=overwrite_output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
        save_strategy = "no"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

    trainer.train()
    trainer.save_model()

In [ ]:
output_dir = 'gpt_rev'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 20
save_steps = 0.5

In [ ]:
train(
    train_dataset = train_dataset,
    tokenizer = tokenizer,
    model = model,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


Step,Training Loss
500,2.233700
1000,2.012400
1500,1.900100
2000,1.770600
2500,1.699900
3000,1.575400
3500,1.517200
4000,1.438500
4500,1.371500
5000,1.326900


In [ ]:
model.save_pretrained("/content/drive/MyDrive/gpt2-latest")

In [16]:
def generate_text(sequence, max_length):
    model_path = "/content/drive/MyDrive/gpt2-latest"
    model = GPT2LMHeadModel.from_pretrained(model_path)
    # tokenizer = load_tokenizer(model_path)

    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=5,
        top_p=0.99,
    )
    return(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [ ]:
df_val.iloc[0]["Text"]

'dog one day difficult time walking took vet vet thought arthritis got pain meds doctor freaked doggy parent started researching online came across product decided give try see would help br br talking amazing product dog back normal activities zipping house mix formula food good go fabulous usually thinks m trying poison give medicine becomes terrible struggle takes problems dog shih tzu mix way helps br br definitely worth try baby struggling'

In [ ]:
generate_text(df_val.iloc[0]["Text"],100)

'dog one day difficult time walking took vet vet thought arthritis got pain meds doctor freaked doggy parent started researching online came across product decided give try see would help br br talking amazing product dog back normal activities zipping house mix formula food good go fabulous usually thinks m trying poison give medicine becomes terrible struggle takes problems dog shih tzu mix way helps br br definitely worth try baby struggling keep weight TL;DR great product dog loves'

# GPT 2

In [ ]:
for data in df_val["Text"]:
  print(data)
  break

dog one day difficult time walking took vet vet thought arthritis got pain meds doctor freaked doggy parent started researching online came across product decided give try see would help br br talking amazing product dog back normal activities zipping house mix formula food good go fabulous usually thinks m trying poison give medicine becomes terrible struggle takes problems dog shih tzu mix way helps br br definitely worth try baby struggling


In [17]:
df_temp = df_val.sample(frac = 0.5, random_state  = 1)

In [18]:
result = []
i_s = []
i = -1
for data in df_temp["Text"]:
    i+=1
    print(i, end = " ")
    # sentence = data.split(" TL;DR ")[0].strip()
    try:
        maxy = len(data.split())
        result.append([generate_text(data,100)])
        i_s.append(i)
    except:
        pass
    if i == 2000:
        break

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [19]:
with open("/content/drive/MyDrive/gen_text.pickle", 'wb') as file:
  pickle.dump(result, file)

In [23]:
# summaries = [[values for values in y_test[:10]]]
print(i_s)
summaries = []
i = -1
for values in df_temp["Summary"]:
    i+=1
    if i not in i_s:
        continue
    else:
        print(i, end = " ")
        summaries.append(values)

# print(len(summaries))
# print(len(summaries[0]))
resulty = []
i=-1
for ri in result:
    i+=1
#     print(ri[0])
    r = ri[0].split('\n')[0]
    try:
        resulty.append(r.split(" TL;DR ")[-1].strip())
    except:
        del summaries[i]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 169, 170, 171, 172, 173, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 196, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 220, 221, 222, 223, 224, 225, 226, 227, 229, 230, 231, 232, 233, 234, 235,

In [21]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 28.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [27]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
# f = open("/kaggle/working/results_dump.txt", 'w')
totalRouge1 = [0,0,0]
totalRouge2 = [0,0,0]
totalRougeL = [0,0,0]
length = 0
for i in range(len(resulty)):
    scores = scorer.score(summaries[i], resulty[i])
    length += 1
    # f.write("Given: "+str(summaries[i]))
    # f.write("\n")
    # f.write("Predicted: "+str(resulty[i]))
    # f.write("\n")
    for key in scores:
        if key == 'rouge1':
            totalRouge1[0] = totalRouge1[0] + scores[key].precision
            totalRouge1[1] = totalRouge1[1] + scores[key].recall
            totalRouge1[2] = totalRouge1[2] + scores[key].fmeasure
        elif key == 'rouge2':
            totalRouge2[0] = totalRouge2[0] + scores[key].precision
            totalRouge2[1] = totalRouge2[1] + scores[key].recall
            totalRouge2[2] = totalRouge2[2] + scores[key].fmeasure
        elif key == 'rougeL':
            totalRougeL[0] = totalRougeL[0] + scores[key].precision
            totalRougeL[1] = totalRougeL[1] + scores[key].recall
            totalRougeL[2] = totalRougeL[2] + scores[key].fmeasure
    #     f.write(f'{key}: {scores[key]}')
    # f.write("\n")
    # f.write("\n")

meanRouge1 = [score/length for score in totalRouge1]
meanRouge2 = [score/length for score in totalRouge2]
meanRougeL = [score/length for score in totalRougeL]
print(length)
print("Total Score: ")
print(meanRouge1)
print(meanRouge2)
print(meanRougeL)


449
Total Score: 
[0.15880294811960785, 0.16046505461872948, 0.13756710844493886]
[0.02935442081887263, 0.03153568777176795, 0.025654143404486223]
[0.15541859141521178, 0.1562466857567081, 0.13450399596269583]


In [30]:
print("=================================================")
print("TOTAL SCORES")
print("=================================================")
print("ROUGE 1")
print("=================================================")
print("Precision :", end = "")
print(meanRouge1[0])
print("Recall :", end = "")
print(meanRouge1[1])
print("F1 Score :", end = "")
print(meanRouge1[2])
print("=================================================")
print("=================================================")
print("ROUGE 2")
print("=================================================")
print("Precision :", end = "")
print(meanRouge2[0])
print("Recall :", end = "")
print(meanRouge2[1])
print("F1 Score :", end = "")
print(meanRouge2[2])
print("=================================================")
print("=================================================")
print("ROUGE L")
print("=================================================")
print("Precision :", end = "")
print(meanRougeL[0])
print("Recall :", end = "")
print(meanRougeL[1])
print("F1 Score :", end = "")
print(meanRougeL[2])
print("=================================================")

TOTAL SCORES
ROUGE 1
Precision :0.15880294811960785
Recall :0.16046505461872948
F1 Score :0.13756710844493886
ROUGE 2
Precision :0.02935442081887263
Recall :0.03153568777176795
F1 Score :0.025654143404486223
ROUGE L
Precision :0.15541859141521178
Recall :0.1562466857567081
F1 Score :0.13450399596269583
